In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class BoardGameNet(nn.Module):
    def __init__(self, board_size, hidden_size=128):
        super(BoardGameNet, self).__init__()
    
        input_layer_size = board_size * board_size + 1 # 1 neuron for every square, and another
                                                     #   neuron to indicate which player is moving 


        self.fc1 = nn.Linear(input_layer_size, hidden_size) # input layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # hidden layer
        self.fc3 = nn.Linear(hidden_size, board_size * board_size) # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        x = F.softmax(x, dim=1)

In [4]:
board_size = 3
model = BoardGameNet(board_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

building the dataloader:

In [7]:
bd = [ [-1, 0, -1],
        [1, -1, 0],
        [-1, 0, -1]
]

def check_winner(board):
    # Check rows, columns, and diagonals for a winner
    for row in board:
        if abs(sum(row)) == 3:  # All squares in row are the same
            return True
    
    for col in range(3):
        if abs(sum([board[row][col] for row in range(3)])) == 3:
            return True
    
    # Check diagonals
    if abs(board[0][0] + board[1][1] + board[2][2]) == 3:
        return True
    if abs(board[0][2] + board[1][1] + board[2][0]) == 3:
        return True
    
    return False

print(check_winner(bd))

True


In [ ]:
PLAYER1, BLANK, PLAYER2 = -1, 0, 1 #enum encoding representation of square values


board = [[0] * 3] # each "board" is 9 squares encoded with x or o as -1 / 1
                    # and an element dictating who's turn it is (player1 or player2) as -1 / 1
                    # am thinking about adding the turn element after populating the board


boards = [] # set of boards

def buildBoards(boards):

    # read from current pool of boards
    # determine player turn ?
    # place a square for every blank space
    # if the board has a winner, then delete it
    # add this new board to the pool of boards


    # assume its player1's turn
    for board in boards:
        board_copy = board

        for square in board_copy:
            if square == BLANK:
                square = PLAYER1

                boards.append(boardCopy)
                square = BLANK






In [5]:
def train(model, data_loader, epochs=10):
    for epoch in range(epochs):
        for board_state, move_label in data_loader:

            #zero gradients
            optimizer.zero_grad()


            #forward pass (propagate forward)
            output = model(board_state)
            loss = criterion(output, move_label)

            
            # Backpropagation and Optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

In [10]:
data = [[ [1], [2] ], [ [-600], [4] ]]

for a, b in data:
    print(a) # board encoding (some list of numbers)
    print(b) # label (some index)



[1]
[2]
[-600]
[4]


In [25]:
# x's are -1
# o's are 1
def printBoard(encoding):

    board = [[0, 0, 0], [0, 0, 0], [0, 0, 0] ]
    counter = 0

    for move in encoding:
        
        if counter == 9:
            

            for row in board:
                print(row)
                print('\n')

            return

        if (move == -1):
            board[int(counter / 3)][counter % 3] = "x"
        elif (move == 0):
            board[int(counter / 3)][counter % 3] = '_'
        else:
            board[int(counter / 3)][counter % 3] = 'o'
        
        counter+=1
        
    
printBoard([-1, -1, 0, 0, 0, 0, 1, 1, 1, 'b'])


['x', 'x', '_']


['_', '_', '_']


['o', 'o', 'o']


